# GEPA: Optimize Anything using ASI (additional side information)

GEPA is a text evolution engine: Given a target metric, GEPA can efficiently search for the right parameters (including numerical, textual and code) to improve that metric. This way, GEPA can optimize essentially represent _anything_ that has a textual representation. In this post, we leverage this insight to present GEPA's optimize-anything API, which leverages the reflective capabilities of LLMs, to optimize anything representable as text. Crucially, GEPA can leverage any additional information available from the optimization environment simply by serializing into text.

### Set up a dataset

In [ ]:
from datasets import load_dataset
import random
import dspy

# Load and process train/val data
all_train = [
    dspy.Example(input=x["problem"], solution=x["solution"], answer=x["answer"]).with_inputs("input")
    for x in load_dataset("AI-MO/aimo-validation-aime", "default", split="train")
]

# Load test data
testset = [
    dspy.Example(input=x["problem"], answer=x["answer"]).with_inputs("input")
    for x in load_dataset("MathArena/aime_2025", "default", split="train")
]

# Shuffle and split
random.Random(0).shuffle(all_train)
mid = len(all_train) // 2
trainset, valset = all_train[:mid], all_train[mid:]

len(trainset), len(valset), len(testset)

(45, 45, 30)